In [174]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import lightgbm as lgb
from sklearn.metrics import roc_auc_score, log_loss

In [175]:
col_df = pd.read_csv('col_list_SNA.csv', index_col=0)
col_list = list(col_df['feature'])[0:200]
col_list.remove("Unnamed: 0")

In [177]:
X_train = pd.read_csv('X_train_sample_x2.csv', index_col =0, usecols = col_list)
X_test = pd.read_csv('X_test_sample.csv', index_col=0, usecols = col_list)

In [11]:
y_train = pd.read_csv('X_train_sample_x2.csv', index_col =0)[['target']]
y_test = pd.read_csv('X_test_sample.csv', index_col =0)[["instanceId_userId", "instanceId_objectId"]]
y_train.head()

,target
10097801,1.0
13545290,0.0
2256020,0.0
25454667,0.0
30238157,0.0


# with images

In [14]:
test_word = pd.read_csv('test_word.csv', index_col=0)

In [15]:
train_word1 = pd.read_csv('train_word1.csv', index_col=0)
train_word410001 = pd.read_csv('train_word410001.csv', index_col=0)
train_word820001 = pd.read_csv('train_word820001.csv', index_col=0)
train_word1230001 = pd.read_csv('train_word1230001.csv', index_col=0)

In [16]:
word = pd.concat([test_word, train_word1, train_word410001, train_word820001, train_word1230001])
word = word.drop_duplicates(subset='ImageId', keep='first')

In [18]:
del test_word, train_word1, train_word410001, train_word820001, train_word1230001

In [19]:
word = word.replace(['green_lizard', 'Komodo_dragon', 'spotted_salamander', \
                     'European_fire_salamander', 'American_alligator','agama',\
                     'American_chameleon', 'Gila_monster', 'alligator_lizard',\
                     'whiptail', 'African_crocodile','common_iguana', 'common_newt',\
                     'axolotl', 'frilled_lizard','African_chameleon', 'banded_gecko',\
                     'eft', 'garter_snake', 'green_snake', 'night_snake', 'king_snake',\
                     'sidewinder', 'hognose_snake', 'sea_snake', 'water_snake', \
                     'boa_constrictor', 'ringneck_snake', 'rock_python', 'green_mamba',\
                     'vine_snake','horned_viper','thunder_snake','Indian_cobra'],'reptile')

In [20]:
word = word.replace(['black_and_gold_garden_spider','wolf_spider','cicada','leafhopper',\
                     'garden_spider','tiger_beetle', 'grasshopper', 'dung_beetle', \
                     'black_widow', 'weevil', 'harvestman', 'flatworm', 'mantis',\
                     'damselfly','leaf_beetle','lycaenid','lacewing','ground_beetle',\
                     'sea_cucumber', 'long-horned_beetle', 'centipede', 'tarantula', \
                     'rhinoceros_beetle', 'fly', 'trilobite', 'cockroach', 'dragonfly',\
                     'ladybug', 'spider_web', 'scorpion', 'apiary', 'tick',\
                     'cabbage_butterfly', 'ant', 'chiton','monarch', 'barn_spider', \
                     'isopod','sulphur_butterfly','nematode','bee','grille'],'insects')

In [21]:
word = word.replace(['red-backed_sandpiper', 'dowitcher', 'American_coot', 'redshank',\
                     'ruddy_turnstone', 'oystercatcher','toucan', 'hornbill', 'junco',\
                     'quail', 'bustard', 'bittern', 'magpie', 'European_gallinule',\
                     'ostrich', 'little_blue_heron', 'limpkin', 'jay', 'coucal', \
                     'chickadee', 'drake','water_ouzel','bulbul','robin','black_swan',\
                     'jacamar', 'vulture', 'peacock', 'sulphur-crested_cockatoo', \
                     'house_finch','red-breasted_merganser', 'white_stork', \
                     'ruffed_grouse', 'spoonbill', 'lorikeet', 'African_grey', \
                     'bee_eater', 'flamingo', 'goldfinch', 'bald_eagle', 'ptarmigan',\
                     'pelican', 'partridge', 'albatross', 'indigo_bunting',\
                     'American_egret','black_stork','goose','macaw','great_grey_owl',\
                     'hummingbird', 'brambling', 'cock', 'hen', 'prairie_chicken', \
                     'black_grouse', 'king_penguin'], 'bird')

In [22]:
word = word.replace(['anemone_fish', 'stingray', 'puffer', 'eel', 'lionfish', \
                     'rock_beauty', 'great_white_shark', 'tiger_shark','hammerhead',\
                     'starfish', 'sea_slug', 'goldfish', 'grey_whale', 'dugong', \
                     'brain_coral', 'sea_anemone', 'hermit_crab', 'spiny_lobster', \
                     'fiddler_crab', 'rock_crab', 'slug', 'sea_urchin', 'crayfish', \
                     'Dungeness_crab','snail','tree_frog','bullfrog','tailed_frog',\
                     'box_turtle', 'mud_turtle', 'loggerhead', 'leatherback_turtle',\
                     'terrapin','sturgeon', 'chambered_nautilus', 'coho', 'king_crab',\
                     'electric_ray','gar','amphibian','conch','tench','killer_whale',\
                     'sea_lion', 'jellyfish', 'American_lobster', 'coral_reef',\
                     'barracouta'], 'sea_creatures')

In [23]:
word = word.replace(['EntleBucher', 'silky_terrier', 'miniature_poodle',\
                     'Tibetan_terrier', 'Shetland_sheepdog',\
                     'Greater_Swiss_Mountain_dog','African_hunting_dog','otterhound',\
                     'miniature_schnauzer','Australian_terrier', 'Norfolk_terrier',\
                     'English_springer', 'Welsh_springer_spaniel', 'basset',\
                     'Bernese_mountain_dog', 'Border_terrier', 'Irish_terrier',\
                     'Appenzeller', 'Kerry_blue_terrier', 'toy_terrier', 'bluetick',\
                     'Blenheim_spaniel', 'Brittany_spaniel', 'Scotch_terrier',\
                     'soft-coated_wheaten_terrier',  'English_foxhound',\
                     'affenpinscher','Bedlington_terrier', 'Irish_water_spaniel',\
                     'dalmatian', 'standard_schnauzer', 'curly-coated_retriever',\
                     'black-and-tan_coonhound','Airedale','giant_schnauzer','briard',\
                     'Shih-Tzu', 'Sealyham_terrier', 'Walker_hound', 'Leonberg',\
                     'English_setter', 'boxer','Scottish_deerhound', 'Rottweiler',\
                     'Pekinese', 'Japanese_spaniel', 'bull_mastiff', \
                     'miniature_pinscher','Boston_bull','Newfoundland','toy_poodle',\
                     'Irish_setter','Italian_greyhound','Border_collie','keeshond',\
                     'cocker_spaniel', 'standard_poodle','Gordon_setter',\
                     'Chesapeake_Bay_retriever','Weimaraner','flat-coated_retriever',\
                     'beagle', 'wire-haired_fox_terrier', 'Rhodesian_ridgeback',\
                     'Ibizan_hound', 'Norwich_terrier', 'basenji', 'Dandie_Dinmont',\
                     'collie', 'Irish_wolfhound', 'Tibetan_mastiff', \
                     'German_short-haired_pointer', 'Lakeland_terrier',\
                     'Brabancon_griffon','Saint_Bernard', 'Doberman','Maltese_dog',\
                     'clumber', 'schipperke', 'Yorkshire_terrier','papillon',\
                     'French_bulldog', 'komondor', 'West_Highland_white_terrier',\
                     'American_Staffordshire_terrier','borzoi', 'Saluki',\
                     'Norwegian_elkhound', 'dingo','Samoyed', 'Eskimo_dog',\
                     'Staffordshire_bullterrier','malinois','Old_English_sheepdog',\
                     'chow', 'Mexican_hairless', 'Pomeranian', 'Great_Pyrenees',\
                     'whippet', 'vizsla', 'pug','Sussex_spaniel', 'Afghan_hound',\
                     'kelpie','malamute','bloodhound','golden_retriever','kuvasz',\
                     'groenendael', 'Bouvier_des_Flandres', 'Labrador_retriever',\
                     'Great_Dane', 'Siberian_husky', 'Chihuahua', 'German_shepherd',\
                     'redbone'], 'dog')

In [24]:
word = word.replace(['Angora', 'Siamese_cat', 'tiger_cat', 'Egyptian_cat', 'tabby', \
                     'Persian_cat'], 'cat')

In [25]:
word = word.replace(['howler_monkey', 'squirrel_monkey', 'spider_monkey', 'colobus',\
                     'siamang', 'langur', 'marmoset', 'three-toed_sloth', 'capuchin',\
                     'gibbon', 'guenon', 'gorilla', 'baboon', 'titi','patas', \
                     'proboscis_monkey', 'indri', 'macaque', 'orangutan', \
                     'chimpanzee'], 'monkey')

In [26]:
word = word.replace(['agaric', 'mushroom', 'bolete', 'stinkhorn', 'gyromitra',\
                     'earthstar', 'coral_fungus', 'hen-of-the-woods'], 'mushrooms')

In [27]:
word = word.replace(['plane', 'space_shuttle', 'airliner', 'parachute', 'warplane', 'airship'], 'air')

In [28]:
word = word.replace(['yawl', 'catamaran', 'lifeboat', 'speedboat', 'trimaran', \
                     'schooner', 'paddlewheel','liner','fireboat', 'canoe', \
                     'container_ship', 'aircraft_carrier', 'gondola', \
                     'drilling_platform', 'submarine'], 'boat')

In [29]:
word = word.replace(['electric_locomotive','freight_car', 'bullet_train', \
                     'steam_locomotive', 'streetcar', \
                     'passenger_car', 'trolleybus'], 'train')

In [30]:
word = word.replace(['impala', 'Model_T', 'convertible', 'pickup', 'jeep', \
                     'minivan', 'sports_car', 'limousine','car_wheel', 'car_mirror',\
                     'seat_belt'], 'car')

In [31]:
word = word.replace(['plow','forklift','harvester','thresher','tractor',\
                     'garbage_truck','crane','moving_van','snowplow','tow_truck',\
                     'trailer_truck','fire_engine','police_van','ambulance',\
                     'school_bus','recreational_vehicle','minibus','cab'], 'work_machines')

In [32]:
word = word.replace(['diamondback', "bicycle-built-for-two", 'moped', 'motor_scooter',\
                     'unicycle', 'mountain_bike', 'snowmobile', 'racer', 'golfcart', \
                     'go-kart'], 'bike')

In [33]:
word = word.replace(['horse_cart', 'oxcart', 'jinrikisha', 'dogsled'], 'cart')

In [34]:
word = word.replace(['maypole', 'totem_pole', 'carousel', 'park_bench', 'fountain',\
                     'swing', 'yurt', ], 'park')

In [35]:
word = word.replace(['Lhasa', 'cliff_dwelling', 'water_tower', 'obelisk', 'tile_roof',\
                     'radio_telescope','triumphal_arch','viaduct','sundial','Pembroke',\
                     'dome', 'stone_wall','suspension_bridge', 'megalith',\
                     'steel_arch_bridge', 'manhole_cover', 'stupa', 'barrow','mosque',\
                     'bell_cote', 'beacon', 'patio', 'solar_dish', 'dam', 'castle', \
                     'palace','guillotine', 'pedestal', 'upright'], 'architecture')

In [36]:
word = word.replace(['valley', 'cliff','alp','seashore','lakeside','geyser','volcano',\
                     'promontory', 'pier','sandbar', 'planetarium', 'dock', 'wreck', \
                     'breakwater', 'boathouse', 'pole'], 'nature')

In [37]:
word = word.replace(['vestment', 'altar', 'church', 'monastery'], 'churchs')

In [38]:
word = word.replace(['cheeseburger', 'broccoli','zucchini', 'espresso','mashed_potato',\
                     'guacamole', 'ice_cream','bagel', 'pizza', 'ice_lolly', 'hotdog',\
                     'acorn_squash', 'burrito', 'pretzel', 'honeycomb', 'dough',\
                     'confectionery', 'carbonara', 'rotisserie', 'chocolate_sauce',\
                     'hot_pot', 'bakery','consomme', 'trifle', 'spaghetti_squash',\
                     'meat_loaf', 'eggnog', 'potpie', 'French_loaf', 'pineapple', \
                     'acorn', 'fig', 'drumstick', 'Granny_Smith', 'artichoke', \
                     'banana', 'orange', 'pomegranate','lemon','jackfruit','buckeye',\
                     'cauliflower', 'butternut_squash','corn','cucumber','bell_pepper',\
                     'custard_apple',  'head_cabbage', 'strawberry'], 'food')

In [39]:
word = word.replace(['beer_bottle', 'wine_bottle', 'beer_glass', 'red_wine', \
                     'whiskey_jug', 'restaurant'], 'alcohol')

In [40]:
word = word.replace(['soup_bowl', 'Crock_Pot', 'teapot', 'saltshaker','ladle','wok',\
                     'strainer', 'pitcher','bottlecap', 'water_bottle', 'coffeepot', \
                     'spatula', 'pop_bottle', 'goblet', 'paddle', 'measuring_cup', \
                     'wooden_spoon', 'corkscrew', 'water_jug', 'mixing_bowl', 
                     'cocktail_shaker','cup', 'frying_pan', 'beaker', 'coffee_mug',\
                     'caldron', 'Dutch_oven', 'tray', 'vase', 'pot',\
                     'plate', 'mortar', 'cleaver', 'plate_rack'], 'dinnerware')

In [41]:
word = word.replace(['cricket','scuba_diver','baseball','golf_ball','football_helmet',\
                     'mountain_tent','pool_table','rugby_ball','snorkel','croquet_ball',\
                     'basketball', 'knee_pad', 'crash_helmet', 'ballplayer', 'racket',\
                     'tennis_ball', 'volleyball', 'ping-pong_ball', 'puck', 'whistle',\
                     'soccer_ball', 'sleeping_bag', 'punching_bag', 'bobsled',\
                     'dumbbell','barbell', 'bow','balance_beam','ski','horizontal_bar',\
                     'parallel_bars'], 'sport')

In [42]:
word = word.replace(['matchstick', 'candle', 'lighter'], 'fire')

In [43]:
word = word.replace(['tiger', 'lynx', 'lion', 'leopard', 'jaguar', 'snow_leopard', 'cougar', 'cheetah'], 'big_cat')

In [44]:
word = word.replace(['timber_wolf', 'white_wolf', 'red_wolf', 'grey_fox', 'red_fox',\
                     'Arctic_fox', 'kit_fox', 'coyote', 'dhole'], 'wolf_fox')

In [45]:
word = word.replace(['brown_bear', 'sloth_bear', 'American_black_bear', 'koala',\
                     'lesser_panda', 'giant_panda','wallaby', 'armadillo', 'skunk',\
                     'badger', 'otter', 'marmot', 'wombat', 'beaver', 'hyena', \
                     'porcupine', 'echidna', 'platypus', 'guinea_pig', 'hamster',\
                     'hare', 'mouse', 'fox_squirrel','mongoose', 'Madagascar_cat',\
                     'black-footed_ferret', 'mink', 'polecat', 'meerkat', \
                     'weasel'], 'wild_animals')

In [46]:
word = word.replace(['wild_boar', 'water_buffalo', 'warthog', 'bighorn', \
                     'African_elephant', 'gazelle', 'bison','zebra', 'llama',\
                     'hippopotamus', 'triceratops', 'hartebeest', 'ibex', 'ox',\
                     'Indian_elephant', 'hog', 'Arabian_camel', 'tusker'], 'hocktail')

In [47]:
word = word.replace(['chiffonier', 'china_cabinet','chest','folding_chair','pillow',\
                     'rocking_chair', 'safe','entertainment_center', 'table_lamp',\
                     'barber_chair', 'shoji', 'sliding_door', 'bannister',\
                     'window_shade', 'fire_screen', 'desk', 'dining_table',\
                     'four-poster', 'wardrobe','studio_couch', 'prayer_rug', \
                     'window_screen', 'throne'], 'furniture')

In [48]:
word = word.replace(['cradle', 'bassinet', 'diaper', 'beach_wagon', 'crib', 'bib'], 'furniture_ch')

In [49]:
word = word.replace(['toilet_tissue', 'apron', 'dishrag', 'plunger', 'mosquito_net',\
                     'doormat', 'broom', 'bath_towel', 'washbasin', 'soap_dispenser',\
                     'tub', 'paper_towel', 'quilt', 'bathtub',\
                     'toilet_seat', 'handkerchief', 'shower_curtain'], 'domestic')

In [50]:
word = word.replace(['space_heater', 'dishwasher', 'toaster', 'scale', 'radiator',\
                     'espresso_maker', 'vacuum','electric_fan', 'sewing_machine',\
                     'waffle_iron', 'washer', 'iron', 'stove', 'microwave', \
                     'refrigerator'], 'domestic_tech')

In [51]:
word = word.replace(['digital_clock', 'digital_watch','magnetic_compass','barometer',\
                     'odometer','parking_meter','stopwatch', 'binoculars', 'hourglass',\
                     'remote_control',  'wall_clock', 'analog_clock', 'switch', \
                     'scoreboard', 'combination_lock', 'vault', 'padlock'], 'appliances')

In [52]:
word = word.replace(['screw', 'hatchet', 'lawn_mower', 'power_drill', 'reel', \
                     'disk_brake', 'hammer', 'spindle', 'chain', 'chain_saw', 'coil',\
                     'screwdriver',"carpenter's_kit",'oil_filter','hand_blower',\
                     'can_opener', 'shovel', 'hook', 'torch', 'mousetrap'], 'household')

In [53]:
word = word.replace(['French_horn', 'trombone', 'cello','acoustic_guitar','oboe',\
                     'cornet', 'violin', 'bassoon','organ', 'grand_piano',\
                     'steel_drum', 'banjo', 'flute', 'drum', 'chime', 'marimba',\
                     'loudspeaker', 'microphone', 'electric_guitar', 'pick', 'sax',\
                     'ocarina', 'accordion', 'maraca', 'gong', 'harmonica', 'panpipe',\
                     'harp','home_theater'], 'musical')

In [54]:
word = word.replace(['Polaroid_camera','computer_keyboard','radio','dial_telephone',\
                     'printer','cassette_player','CD_player','tripod','photocopier',\
                     'reflex_camera','typewriter_keyboard','joystick','tape_player',\
                     'projector','lens_cap', 'laptop','iPod', 'desktop_computer', \
                     'hard_disc','cassette','notebook','space_bar','modem',\
                     'pay-phone','hand-held_computer','cellular_telephone',\
                     'monitor', 'screen', 'television', 'ram'], 'technique')

In [55]:
word = word.replace(['pencil_box','abacus','fountain_pen','file','pencil_sharpener',\
                     'rubber_eraser', 'safety_pin', 'ballpoint', 'loupe', 'rule',\
                     'paintbrush', 'quill', 'slide_rule', 'thimble', 'binder',\
                     'letter_opener','envelope','bookcase', 'library','bookshop',\
                     'maze', 'jigsaw_puzzle', 'crossword_puzzle'], 'office')

In [56]:
word = word.replace(['ringlet', 'hair_slide', 'ear', 'wig'], 'head')

In [57]:
word = word.replace(["yellow_lady's_slipper", 'rapeseed', 'cardoon', 'daisy', 'sorrel'], 'flowers')

In [58]:
word = word.replace(['cannon', 'revolver', 'missile', 'half_track','assault_rifle',\
                     'rifle', 'tank', 'projectile', 'scabbard'], 'military')

In [59]:
word = word.replace(['piggy_bank', "jack-o'-lantern", 'kite', 'wood_rabbit', \
                     'pinwheel', 'Christmas_stocking',\
                     'ice_bear', 'bubble', 'teddy', 'tricycle', 'balloon'], 'toy')

In [60]:
word = word.replace(['syringe', 'stethoscope', 'oscilloscope', 'Petri_dish', \
                     'crutch', 'hip', 'stretcher', 'swab', 'medicine_chest',\
                     'oxygen_mask', 'neck_brace', 'pill_bottle', 'Band_Aid', \
                     'lab_coat'], 'medicine')

In [61]:
word = word.replace(['backpack', 'wallet', 'purse', 'mailbag'], 'bags')

In [62]:
word = word.replace(['mitten', 'Cardigan', 'sock', 'sweatshirt', 'trench_coat', \
                     'velvet', 'cardigan', 'wool', 'poncho', 'kimono','jersey', \
                     'stole', 'academic_gown', 'cloak', 'fur_coat', 'pajama',\
                     'jean', 'miniskirt'], 'clothes')

In [63]:
word = word.replace(['sarong', 'swimming_trunks', 'sunglass', 'sunglasses', \
                     'nipple', 'brassiere', 'bikini', 'maillot'], 'beach')

In [64]:
word = word.replace(['holster', 'bulletproof_vest', 'gasmask', 'military_uniform',\
                     'scabbard','ski_mask','mask','admiral','pirate','chain_mail',\
                     'breastplate', 'cuirass', 'pickelhaube', 'shield'], 'camouflage')

In [65]:
word = word.replace(['running_shoe', 'Loafer', 'cowboy_boot', 'sandal', 'clog'], 'shoes')

In [66]:
word = word.replace(['mortarboard', 'cowboy_hat', 'sombrero', 'bathing_cap', \
                     'shower_cap', 'bonnet', 'bearskin'], 'hats')

In [67]:
word = word.replace(['vending_machine', 'bookshop', 'gas_pump', 'shoe_shop',\
                     'grocery_store', 'butcher_shop', 'tobacco_shop', 'hamper',\
                     'barbershop', 'slot', 'cash_machine', 'toyshop'], 'shop')

In [68]:
word = word.replace(['groom', 'suit', 'Windsor_tie', 'bow_tie', 'bolo_tie', 'buckle'], 'men')

In [69]:
word = word.replace(['abaya', 'overskirt', 'gown', 'hoopskirt', 'feather_boa', 'necklace'], 'women')

In [70]:
word = word.replace(['perfume', 'lipstick', 'hair_spray', 'face_powder', 'sunscreen',\
                     'lotion', 'nail'], 'cosmetics')

In [71]:
word = word.replace(['theater_curtain', 'stage', 'cinema', 'spotlight'], 'stages')

In [72]:
word = word.replace(['plastic_bag', 'crate', 'carton', 'packet', 'rain_barrel', 'milk_can', 'bucket', 'barrel', 'ashcan', 'hamper'], 'packets')

In [73]:
word = word.replace(['hay', 'thatch', 'birdhouse', 'worm_fence', 'greenhouse', 'barn', 'lumbermill', 'picket_fence', 'mobile_home', 'chainlink_fence', 'prison'], 'village')

In [77]:
rare_Image = word['ImageLable'].value_counts()[(word['ImageLable'].value_counts() < 1000) ==True].index
print((word['ImageLable'].nunique()) - (rare_Image.nunique()))

66


In [78]:
word['ImageLable'] = word['ImageLable'].apply(lambda x: 'RARE' if x in rare_Image else x)
word['ImageLable_1'] = word['ImageLable_1'].apply(lambda x: 'RARE' if x in rare_Image else x)
word['ImageLable_2'] = word['ImageLable_2'].apply(lambda x: 'RARE' if x in rare_Image else x)
word['ImageLables'] = word['ImageLable'] + ' ' + word['ImageLable_1'] + ' ' + word['ImageLable_2']

In [80]:
%%time
temp_word = word['ImageLables'].astype(str).str.get_dummies(" ") 

CPU times: user 7min 11s, sys: 14.1 s, total: 7min 25s
Wall time: 7min 25s


In [81]:
word = word.drop(['ImageLables', 'ImageLable', 'ImageLable_1', 'ImageLable_2'], axis=1)

In [83]:
X_word = pd.concat([word, temp_word], axis=1, sort=False)
X_word.shape, word.shape, temp_word.shape

((1920452, 68), (1920452, 1), (1920452, 67))

In [84]:
del word, temp_word

In [85]:
X_train['target'] = y_train['target']
X_test['target'] = -1

In [89]:
X_train['ImageId']=pd.read_csv('X_train_sample_x2.csv', index_col =0)[['ImageId']]
X_test['ImageId']=pd.read_csv('X_train_sample_x2.csv', index_col =0)[['ImageId']]

In [91]:
X_all = pd.concat([pd.DataFrame(X_train), pd.DataFrame(X_test)], sort=False)
X_ = X_all.merge(X_word, on='ImageId', how='left', sort=False)

In [93]:
del X_all, #X_word

In [94]:
X_train_ = X_[(X_.target != -1)].drop(['target', 'ImageId'], axis=1)
X_test_ = X_[(X_.target == -1)].drop(['target', 'ImageId'], axis=1)

In [96]:
set(X_train.columns) - set(X_test.columns), set(X_test.columns) - set(X_train.columns)

(set(), set())

In [97]:
del X_

In [98]:
set(X_train_.columns) - set(X_test_.columns), set(X_test_.columns) - set(X_train_.columns)

(set(), set())

In [100]:
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
X_train.head()

,instanceId_userId,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,...,audit_resourceType_14,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,target,ImageId
0,11880416,19834498,1,1518943264076,681760,1518785445000,0,1,1,0,...,0,0,0,0,0,0,0,1,0.0,['5bfca7c32d3c92835e3209575e285c8c']
1,8059910,27763185,5,1519399376900,237871,1519390992000,0,0,0,0,...,1,0,0,0,0,1,0,0,NaN,NaN
2,10389428,25540668,9,1517737977459,907314,1517687714000,0,1,0,918,...,0,0,0,0,0,0,0,1,NaN,NaN
3,11758940,21660296,2,1520944495639,65634961,1520857023000,0,1,0,31,...,0,0,1,0,0,0,0,0,NaN,NaN
4,14951459,16783446,3,1521617771342,1521617771342,1521580513000,0,1,0,65,...,0,0,0,1,0,0,0,0,NaN,NaN


In [101]:
X_test.head()

,instanceId_userId,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,...,audit_resourceType_14,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6,target,ImageId
0,602,25708174,0,1521871398528,3571061,1521828068000,0,6,0,26,...,0,0,0,0,0,0,1,0,-1,NaN
1,602,24393746,7,1521923900981,3271210,1521915281000,0,10,0,55,...,0,0,0,0,0,0,1,0,-1,NaN
2,803,25400331,2,1522127577706,142606,1521943280000,0,1,0,1491,...,0,0,1,0,0,0,0,0,-1,NaN
3,803,36083673,20,1522258130090,184593,1522249825000,0,1,0,937,...,0,0,0,1,0,0,0,0,-1,NaN
4,803,33713302,0,1522420497473,1522420497473,1522407014000,0,2,0,408,...,0,0,0,0,0,1,0,0,-1,NaN


In [102]:
X_train_ = X_train_.fillna(0)
X_test_ = X_test_.fillna(0)
X_train_.head()

,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,...,technique,toy,train,umbrella,village,web_site,wild_animals,wolf_fox,women,work_machines
0,19834498,1,1518943264076,681760,1518785445000,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,27763185,5,1519399376900,237871,1519390992000,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,25540668,9,1517737977459,907314,1517687714000,0,1,0,918,133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,21660296,2,1520944495639,65634961,1520857023000,0,1,0,31,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,16783446,3,1521617771342,1521617771342,1521580513000,0,1,0,65,10,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#  text

In [103]:
import pyarrow.parquet as parquet
# Where the downloaded data are
input_path = '/mnt/data/dataset/'
# Where to store results
output_path = './'

In [104]:
content_train = parquet.read_table(input_path + 'texts/train/', columns = ['objectId', 'text']).to_pandas()
content_train.rename(columns = {'objectId':'instanceId_objectId'}, inplace = True)
content_train.head(3)

,instanceId_objectId,text
0,2695,Деревня Машканка.Автор Швецова З Н . Малая Род...
1,92687,Классно танцуют Присоединяйтесь к нашей группе...
2,92851,"A-Dessa - Женщины, я не танцую Присоединяйтесь..."


In [105]:
#Загружаем содаржание постов из паркета
content_test = parquet.read_table(input_path + 'texts/test/', columns = ['objectId', 'text']).to_pandas()
content_test.rename(columns = {'objectId':'instanceId_objectId'}, inplace = True)
content_test.head(3)

,instanceId_objectId,text
0,362378,С таким и не поспоришь..)))))
1,362665,Волонтер спасла нигерийского мальчика
2,362903,Стаффтерьер накормил котенка


In [106]:
content_train_1 = content_train[['instanceId_objectId', 'text']].drop_duplicates(subset='instanceId_objectId', keep='first')

In [107]:
content_test_1 = content_test[['instanceId_objectId', 'text']].drop_duplicates(subset='instanceId_objectId', keep='first')

In [108]:
content_train.shape, content_test.shape, content_train_1.shape, content_test_1.shape

((3241322, 2), (563660, 2), (3167029, 2), (554901, 2))

In [109]:
#соединяем трейн с постами
X_train_c = X_train_.merge(content_train_1, on='instanceId_objectId', how='left', sort=False)
X_train_c.head(3)

,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,...,toy,train,umbrella,village,web_site,wild_animals,wolf_fox,women,work_machines,text
0,19834498,1,1518943264076,681760,1518785445000,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Флаг какой страны изображен на фото ??? ТАДЖИК...
1,27763185,5,1519399376900,237871,1519390992000,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,25540668,9,1517737977459,907314,1517687714000,0,1,0,918,133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [110]:
X_test_c = X_test_.merge(content_test_1, on='instanceId_objectId', how='left', sort=False)
X_test_c.head(3)

,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,...,toy,train,umbrella,village,web_site,wild_animals,wolf_fox,women,work_machines,text
0,25708174,0,1521871398528,3571061,1521828068000,0,6,0,26,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,24393746,7,1521923900981,3271210,1521915281000,0,10,0,55,11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Потратил на это 8 дней мне 20 лет! Оцените мою...
2,25400331,2,1522127577706,142606,1521943280000,0,1,0,1491,215,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [111]:
X_train_c['text'].fillna(' ', inplace = True)
X_test_c['text'].fillna(' ', inplace = True)
X_train_c.head(3)

,instanceId_objectId,audit_pos,audit_timestamp,audit_timePassed,metadata_createdAt,metadata_numCompanions,metadata_numPhotos,metadata_numPolls,metadata_numSymbols,metadata_numTokens,...,toy,train,umbrella,village,web_site,wild_animals,wolf_fox,women,work_machines,text
0,19834498,1,1518943264076,681760,1518785445000,0,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Флаг какой страны изображен на фото ??? ТАДЖИК...
1,27763185,5,1519399376900,237871,1519390992000,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,25540668,9,1517737977459,907314,1517687714000,0,1,0,918,133,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [114]:
from scipy.sparse import csr_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hackathon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/hackathon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [116]:
stemmer = SnowballStemmer("russian")
stopwords = nltk.corpus.stopwords.words('russian')

In [118]:
tfidfn = TfidfVectorizer(ngram_range=(1, 3), max_df=0.9, min_df=0.01, max_features=400, stop_words=stopwords, 
                         sublinear_tf=True)

In [123]:
%%time
X_train_sparse0 = tfidfn.fit(content_train_1['text'])

CPU times: user 23min 48s, sys: 2min 49s, total: 26min 38s
Wall time: 26min 43s


In [125]:
X_train.shape, X_train_.shape, X_train_c.shape, #X_train_sparse1.shape

((6157569, 141), (6157569, 205), (6157569, 206))

In [126]:
%%time
#X_train_sparse1 = tfidfn.fit_transform(X_train_c['text'])
X_train_sparse1 = tfidfn.transform(X_train_c['text'])

CPU times: user 11min 12s, sys: 11.2 s, total: 11min 24s
Wall time: 11min 30s


In [127]:
%%time
X_test_sparse1 = tfidfn.transform(X_test_c['text'])

CPU times: user 6min 5s, sys: 6.44 s, total: 6min 11s
Wall time: 6min 13s


In [128]:
X_train_x = X_train.drop(['target', 'ImageId'], axis=1)
X_test_x = X_test.drop(['target', 'ImageId'], axis=1)

In [129]:
X_train_sparse = csr_matrix(hstack([X_train_x, X_train_sparse1]))
X_test_sparse = csr_matrix(hstack([X_test_x, X_test_sparse1]))

# model

In [131]:
X_train_1, X_hold, Y_train_1, Y_hold = train_test_split(X_train_sparse, y_train, test_size=0.3, 
                                                        random_state=42, shuffle = True)
X_train_0, X_valid, Y_train_0, Y_valid = train_test_split(X_train_1, Y_train_1, test_size=0.3, 
                                                          random_state=42, shuffle = True)

In [132]:
lgb_train = lgb.Dataset(X_train_1, label=Y_train_1, free_raw_data=False)
lgb_test = lgb.Dataset(X_valid, label=Y_valid, free_raw_data=False)

In [134]:
%%time
model = lgb.train(
                lgb_params,
                lgb_train,
                valid_sets=[lgb_train, lgb_test],
                valid_names=['train', 'test'],
                num_boost_round=5000,
                early_stopping_rounds= 100,
                verbose_eval=100
                )

Training until validation scores don't improve for 100 rounds.
[100]	train's binary_logloss: 0.411593	test's binary_logloss: 0.411799
[200]	train's binary_logloss: 0.409708	test's binary_logloss: 0.409901
[300]	train's binary_logloss: 0.408842	test's binary_logloss: 0.409045
[400]	train's binary_logloss: 0.408242	test's binary_logloss: 0.408437
[500]	train's binary_logloss: 0.407698	test's binary_logloss: 0.407894
[600]	train's binary_logloss: 0.407197	test's binary_logloss: 0.407388
[700]	train's binary_logloss: 0.406742	test's binary_logloss: 0.406936
[800]	train's binary_logloss: 0.40636	test's binary_logloss: 0.406554
[900]	train's binary_logloss: 0.405953	test's binary_logloss: 0.406143
[1000]	train's binary_logloss: 0.405539	test's binary_logloss: 0.405723
[1100]	train's binary_logloss: 0.405195	test's binary_logloss: 0.405377
[1200]	train's binary_logloss: 0.404842	test's binary_logloss: 0.405023
[1300]	train's binary_logloss: 0.40448	test's binary_logloss: 0.404662
[1400]	train

In [135]:
print(roc_auc_score(Y_hold, model.predict(X_hold)))
print(log_loss(Y_hold, model.predict(X_hold)))

0.774716144220658
0.40766893637690726


In [156]:
prediction = model.predict(X_test_sparse, num_iteration=model.best_iteration)
y_test["predictions"]= -prediction

In [158]:
# Peek only needed columns and sort
result = y_test[["instanceId_userId", "instanceId_objectId", "predictions"]].sort_values(
    by=['instanceId_userId', 'predictions'])
result.head()

,instanceId_userId,instanceId_objectId,predictions
381655,8,38659430,-0.412611
381652,8,20989661,-0.398381
381651,8,14385580,-0.310108
381648,8,26815124,-0.274924
381653,8,22160290,-0.242467


In [159]:
# Collect predictions for each user
submit = result.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head()

instanceId_userId
8      [38659430, 20989661, 14385580, 26815124, 22160...
59                        [23685237, 39141251, 29096263]
92                        [12367356, 19818572, 23774973]
107    [20658574, 38038671, 10501436, 20368797, 22664...
158                       [30223730, 33303760, 19280083]
Name: instanceId_objectId, dtype: object

In [160]:
# Persist the first submit
submit.to_csv("collabSubmit8.csv.gz", header = False, compression='gzip')

In [161]:
y_test = y_test.to_csv("y_test_all_2.csv")

In [131]:
y_test = y_test.drop(['predictions'], axis=1)

In [188]:
lgb_params = {
    'task': 'train',
    'learning_rate': 0.1, 
    'max_depth': 9, 
#    'n_estimators': 500, 
    'num_leaves': 20, 
    'reg_alpha': 5.79,    
    'reg_lambda': 3.79,
    'objective': 'binary',
    'metric': 'binary_logloss',
    'random_state': 42,
    'colsample_bytree': 0.9, 
    'subsample': 0.9,
    'n_jobs': 24,
#    'device': 'gpu',   
}

In [189]:
df_train = X_train
df_test = X_test
target = y_train
features = list(df_train)
categorical_feats = None
param = lgb_params
num_folds = 5

In [190]:
%%time
folds = KFold(n_splits= num_folds, shuffle=True, random_state=42)
oof = np.zeros(len(df_train))
predictions = np.zeros(len(df_test))

#start = time.time()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train, target)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = lgb.Dataset(df_train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(df_train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 4000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(df_train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    print('Fold %2d roc_auc : %.6f' % (fold_ + 1, roc_auc_score(target, oof)))    
    #Прогноз
    predictions += clf.predict(df_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.411569	valid_1's binary_logloss: 0.411918
[200]	training's binary_logloss: 0.409756	valid_1's binary_logloss: 0.41035
[300]	training's binary_logloss: 0.408833	valid_1's binary_logloss: 0.409689
[400]	training's binary_logloss: 0.408263	valid_1's binary_logloss: 0.409383
[500]	training's binary_logloss: 0.407808	valid_1's binary_logloss: 0.409194
[600]	training's binary_logloss: 0.407413	valid_1's binary_logloss: 0.409047
[700]	training's binary_logloss: 0.406998	valid_1's binary_logloss: 0.408895
[800]	training's binary_logloss: 0.406639	valid_1's binary_logloss: 0.40879
[900]	training's binary_logloss: 0.406231	valid_1's binary_logloss: 0.408649
[1000]	training's binary_logloss: 0.405779	valid_1's binary_logloss: 0.408464
[1100]	training's binary_logloss: 0.40542	valid_1's binary_logloss: 0.408364
[1200]	training's binary_logloss: 0.405021	valid_1's binary_logloss: 0.40823
[1300

[2900]	training's binary_logloss: 0.399755	valid_1's binary_logloss: 0.408192
[3000]	training's binary_logloss: 0.3995	valid_1's binary_logloss: 0.408189
[3100]	training's binary_logloss: 0.399245	valid_1's binary_logloss: 0.408183
[3200]	training's binary_logloss: 0.398996	valid_1's binary_logloss: 0.408174
[3300]	training's binary_logloss: 0.398719	valid_1's binary_logloss: 0.40815
[3400]	training's binary_logloss: 0.398443	valid_1's binary_logloss: 0.408141
[3500]	training's binary_logloss: 0.398197	valid_1's binary_logloss: 0.40814
Early stopping, best iteration is:
[3451]	training's binary_logloss: 0.398316	valid_1's binary_logloss: 0.408136
Fold  3 roc_auc : 0.599335
fold n°4
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.411677	valid_1's binary_logloss: 0.411811
[200]	training's binary_logloss: 0.409816	valid_1's binary_logloss: 0.410173
[300]	training's binary_logloss: 0.409018	valid_1's binary_logloss: 0.409612
[400]	training'

In [195]:
y_test["predictions"]= -predictions

In [196]:
# Peek only needed columns and sort
result = y_test[["instanceId_userId", "instanceId_objectId", "predictions"]].sort_values(
    by=['instanceId_userId', 'predictions'])
result.head()

,instanceId_userId,instanceId_objectId,predictions
381655,8,38659430,-0.421674
381652,8,20989661,-0.393939
381651,8,14385580,-0.365560
381648,8,26815124,-0.303013
381653,8,22160290,-0.222386


In [197]:
# Collect predictions for each user
submit = result.groupby("instanceId_userId")['instanceId_objectId'].apply(list)
submit.head()

instanceId_userId
8      [38659430, 20989661, 14385580, 26815124, 22160...
59                        [23685237, 29096263, 39141251]
92                        [12367356, 19818572, 23774973]
107    [20658574, 10501436, 20368797, 38038671, 39090...
158                       [30223730, 33303760, 19280083]
Name: instanceId_objectId, dtype: object

In [198]:
submit.to_csv("collabSubmit10.csv.gz", header = False, compression='gzip')

In [199]:
y_test = y_test.to_csv("y_test_all_4.csv")